In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

import pandas,numpy,random
import matplotlib.pyplot as plt

In [ ]:
class MnistDataset(Dataset):
    
    def __init__(self, csv_file):
        self.data_df = pandas.read_csv(csv_file, header=None)
        pass
    
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, index):
        # image target (label)
        label = self.data_df.iloc[index,0]
        target = torch.zeros((10))
        target[label] = 1.0
        
        # image data, normalised from 0-255 to 0-1
        image_values = torch.FloatTensor(self.data_df.iloc[index,1:].values) / 255.0
        
        # return label, image data tensor and target tensor
        return label, image_values, target
    
    def plot_image(self, index):
        img = self.data_df.iloc[index,1:].values.reshape(28,28)
        plt.title("label = " + str(self.data_df.iloc[index,0]))
        plt.imshow(img, interpolation='none', cmap='Blues')
        pass
    
    pass

In [ ]:
mnist_dataset = MnistDataset('mount/My Drive/Colab Notebooks/myo_gan/mnist_data/mnist_train.csv')

In [ ]:
def generate_random_image(size):
  random_data = torch.rand(size)
  return random_data

def generate_random_see(size):
  random_data = torch.randn(size)
  return random_data

def generate_random_one_hot(size):
  label_tensor = torch.zeros((size))
  random_idx =random.randint(0,size-1)
  label_tensor[random_idx] = 1.0
  return label_tensor


In [ ]:
class Discirminator(nn.Module):

  def__init__(self):
    super().__init__()

    self.model = nn.Sequential(
        nn.Linear(784+10,200),
        nn.LeakyReLU(0.02)

        nn.LayerNorm(200),

        nn.Linea(200,1),
        nn.Sigmoid()
    )

    self.loss_function = nn.BCELoss()
    self.optimiser = torch.optim.Adam(self.parameters(), lr=0.0001 )

    self.counter = 0; 
    self.progress = []

    pass


  def forward(self,image_tensor , label_tensor):
    inputs = torch.cat((image_tensor, label_tensor)) 
    return self.moel(inputs)


  def train(self, inputs, label_tensor, targets):
    outputs = self.forward(inputs,label_tensor)
    loss = self.loss_function(outputs, targets)

    self.counter += 1;
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
            pass
        if (self.counter % 10000 == 0):
            print("counter = ", self.counter)
            pass

    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()

    pass

  def plot_progress(self):
        df = pandas.DataFrame(self.progress, columns=['loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
        pass
    
    pass


In [ ]:
%%time
D = Discriminator()

for label, image_data_tensor, label_tensor in mnist_dataset:
    D.train(image_data_tensor, label_tensor, torch.FloatTensor([1.0]))
    D.train(generate_random_image(784), generate_random_one_hot(10), torch.FloatTensor([0.0]))
    pass

In [ ]:
D.plot_progress()

In [ ]:
class Generator(nn.Module):
  def__init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(100+10,200),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(200),
        nn.Linear(200,784),
        nn.Sigmoid()
    )

    self.optimiser = torch.optim.Adam(self.parameters(),lr=0.0001)

    self.counter=0;
    self.progress =[]

    pass

  def forward(self, seed_tensor,label_tensor):
    inputs = torch.cat((seed_tensor,label_tensor))
    return self.model(inputs)

  def train(self,D,inputs,label_tensor,targets):
    g_output = self.forward(inputs,label_tensor)
    d_output = D.forward(g_output,targets)

    loss = D.loss_function(d_output, targets)

    self.counter += 1;
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
            pass

    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()

    pass

   def plot_images(self, label):
        label_tensor = torch.zeros((10))
        label_tensor[label] = 1.0
        f, axarr = plt.subplots(2,3, figsize=(16,8))
        for i in range(2):
            for j in range(3):
                axarr[i,j].imshow(G.forward(generate_random_seed(100), label_tensor).detach().cpu().numpy().reshape(28,28), interpolation='none', cmap='Blues')
                pass
            pass
        pass
    
    def plot_progress(self):
        df = pandas.DataFrame(self.progress, columns=['loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
        pass
    
    pass


In [ ]:
G = Generator()

output = G.forward(generate_random_seed(100), generate_random_one_hot(10))

img = output.detach().numpy().reshape(28,28)

plt.imshow(img, interpolation='none', cmap='Blues')

In [ ]:
D = Discriminator()
G = Generator()

In [ ]:
%%time 

epochs = 12

for epoch in range(epochs):
  print ("epoch = ", epoch + 1)

  for label, image_data_tensor, label_tensor in mnist_dataset:
    D.train(image_data_tensor, label_tensor, torch.FloatTensor([1.0]))
    random_label = generate_random_one_hot(10)
    D.train(G.forward(generate_random_seed(100), random_label).detach(), random_label, torch.FloatTensor([0.0]))
    
    random_label = generate_random_one_hot(10)
    G.train(D, generate_random_seed(100), random_label, torch.FloatTensor([1.0]))

    pass
    
  pass

In [ ]:
D.plot_progress()

In [ ]:
G.plot_progress()

In [ ]:
G.plot_images(9)